# Preparation
## Paths

In [ ]:
# Import Operating System
import os

In [ ]:
ipynb_path = os.path.dirname(os.path.realpath("__file__"))
print(ipynb_path)

In [ ]:
root = "\\".join(ipynb_path.split("\\")[:-2])+"\\datasets"
print(root)

In [ ]:
workspace = "custom"

In [ ]:
paths = {
    'WORKSPACE_PATH': os.path.join(root, workspace),
    'OVERLAYS_PATH': os.path.join(root, workspace,'overlays'),
    'BACKGROUNDS_PATH': os.path.join(root, workspace,'backgrounds'),
    'IMAGES_PATH': os.path.join(root, workspace,'images'),
    'ANNOTATIONS_PATH': os.path.join(root, workspace,'annotations'),
 }
print(paths)

In [ ]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

## Backgrounds

In [ ]:
_, _, background_files = next(os.walk(paths['BACKGROUNDS_PATH']), (None, [], []))
print("background files: " + str(len(background_files)))

## Overlays

In [ ]:
_, overlays_folders, overlays_files = next(os.walk(paths['OVERLAYS_PATH']), (None, [], []))
print("overlays folders: " + str(len(overlays_folders)))
print("overlays files: " + str(len(overlays_files)))

## Labels

In [ ]:
labels = []
overlays = []
for folder in overlays_folders:
    #print("label: " + folder)
    labels.append(folder)
    _, _, overlays_files_versions = next(os.walk(os.path.join(paths['OVERLAYS_PATH'], folder)), (None, [], []))
    for file in overlays_files_versions:
        #print("file: " + file)
        overlays.append(os.path.join(paths['OVERLAYS_PATH'], folder, file))

for file in overlays_files:
    overlays.append(os.path.join(paths['OVERLAYS_PATH'], file))
    file_label = "-".join(file.split("-")[:-1])
    #print("label: " + file_label)
    labels.append(file_label)

#print(labels)
print(overlays)

list_filename = os.path.join(paths['WORKSPACE_PATH'], 'className.list')
#print(list_filename)

with open(list_filename, 'w') as file :
    file.write('')

for (l, label) in enumerate(labels):
    with open(list_filename, 'a') as file :
        file.write(label+"\n")

In [ ]:
#!pip install opencv-python
#!pip install matplotlib
#!pip install scipy
#!pip install imutils

In [ ]:
import cv2
import imutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import math
import random

In [ ]:
img = mpimg.imread(overlays[0])
#print(img)
#imgplot = plt.imshow(img)
y, x = img[:, :, 3].nonzero()
minx = np.min(x)
miny = np.min(y)
maxx = np.max(x)
maxy = np.max(y)
difx = maxx - minx
dify = maxy - miny
dia = int(math.sqrt(math.pow(difx, 2) + math.pow(dify, 2)))
print(dia)
#cropImg = img[miny:maxy, minx:maxx]
cropImg = img[miny:maxy, minx:maxx]
#imgplot = plt.imshow(cropImg)

n_channels = 4
transparent_img = np.zeros((dia, dia, n_channels), dtype=np.single)

overlay_height, overlay_width = cropImg.shape[:2]
# background_padding_y
bpy = int((dia - overlay_height)/2)
# background_padding_x
bpx = int((dia - overlay_width)/2)

transparent_img[bpy:(bpy+overlay_height), bpx:(bpx+overlay_width)] = cropImg

# rotation
rmin = 0
rmax = 45
deg = random.uniform(rmin, rmax)
pos = 2*(random.random()-0.5)
rotated = imutils.rotate(transparent_img, angle=deg*pos)
a = rotated[:, :, 3]

#imgplot = plt.imshow(rotated)

gmin=0.5
gmax=1.5
smin=0.5
smax=1.0

saturation = random.uniform(smin, smax)
gamma = random.uniform(gmin, gmax)

hsv = cv2.cvtColor(rotated, cv2.COLOR_BGR2HSV)

#imgplot = plt.imshow(hsv)

h, s, v = cv2.split(hsv)

# adjust
s_desat = cv2.multiply(s, saturation)
v_gamma = cv2.multiply(v, gamma)
hsv_new = cv2.merge([h, s_desat, v_gamma])

#imgplot = plt.imshow(hsv_new)

# convert to bgr
bgr_adjust = cv2.cvtColor(hsv_new, cv2.COLOR_HSV2BGR)

adjust = cv2.cvtColor(bgr_adjust, cv2.COLOR_RGB2RGBA)
#imgplot = plt.imshow(bgr_adjust)
adjust[:, :, 3] = a

#resize
zmin=0
zmax=50
resize = random.uniform(zmin, zmax)

procent = 1-resize/100
height, width = adjust.shape[:2]
dim = (int(width*procent), height)
resized = cv2.resize(adjust, dim, interpolation=cv2.INTER_AREA)

resized*=255
resized = resized.astype(int)
resized[resized < 0] = 0
resized[resized > 255] = 255

y_, x_ = resized[:, :, 3].nonzero()
minx_ = np.min(x_)
miny_ = np.min(y_)
maxx_ = np.max(x_)
maxy_ = np.max(y_)
cropImg_ = resized[miny_:maxy_, minx_:maxx_]

imgplot = plt.imshow(cropImg_)
#print(cropImg_)